# Token classification

creating lables

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from read_data import read_data, tags_list  # noqa: E402
from training import get_model,get_dataset

/home/mhmmd/neshan/final_project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading test dataset

In [5]:
data = read_data("data/merged.csv")
len(data)

52

In [3]:
file_path = "data/merged.csv"
model_id = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer, model = get_model(model_id)
sample = get_dataset(file_path,tokenizer)

/home/mhmmd/neshan/final_project/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 

## Preprocess

In [19]:
from transformers import AutoTokenizer

model_id = "HooshvareLab/bert-base-parsbert-uncased"
# model_id = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/ham/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

In [20]:
example = sample["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'سردرد',
 'و',
 'سرگیجه',
 'دارم',
 'یک',
 'دکتر',
 'تو',
 'سناباد',
 'میخوام',
 '[SEP]']

However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(
            batch_index=i
        )  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif (
                word_idx != previous_word_idx
            ):  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
tokenized_data = sample.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 28/28 [00:00<00:00, 4231.18 examples/s]


Map: 100%|██████████| 28/28 [00:00<00:00, 5639.13 examples/s]


Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [23]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

In [24]:
import evaluate

seqeval = evaluate.load("seqeval")

Get the NER labels first, and then create a function that passes your true predictions and true labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the scores:

In [25]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tags_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [26]:
id2label = {i: label for i, label in enumerate(tags_list)}
label2id = {v: k for k, v in id2label.items()}
id2label

{0: 'B-problem',
 1: 'I-problem',
 2: 'B-city',
 3: 'I-city',
 4: 'B-expertise',
 5: 'I-expertise',
 6: 'B-gender',
 7: 'I-gender',
 8: 'O'}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [27]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

model = AutoModelForTokenClassification.from_pretrained(
    model_id,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)
for param in model.parameters():
    param.data = param.data.contiguous()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import os

os.environ["MLFLOW_EXPERIMENT_NAME"] = "slot-filling-model"
os.environ["MLFLOW_EXPERIMENT_NAME"] = "slot-filling-model"
# os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
# os.environ["HF_MLFLOW_LOG_ARTIFACTS"]="1"

In [45]:
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="mlflow",
    run_name=f"{model_id}",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

  5%|▌         | 2/40 [00:00<00:02, 16.77it/s]

{'loss': 0.8358, 'grad_norm': 4.911033630371094, 'learning_rate': 1.95e-05, 'epoch': 0.5}
{'loss': 1.0147, 'grad_norm': 3.4215714931488037, 'learning_rate': 1.9e-05, 'epoch': 1.0}


/home/ham/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ham/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                              
  5%|▌         | 2/40 [00:00<00:02, 13.40it/s]

{'eval_loss': 0.7611289620399475, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.7705479452054794, 'eval_runtime': 0.0239, 'eval_samples_per_second': 1169.808, 'eval_steps_per_second': 83.558, 'epoch': 1.0}


 10%|█         | 4/40 [00:01<00:21,  1.68it/s]

...

{'eval_loss': 0.049227528274059296, 'eval_precision': 0.9772727272727273, 'eval_recall': 0.9555555555555556, 'eval_f1': 0.9662921348314608, 'eval_accuracy': 0.9965753424657534, 'eval_runtime': 0.024, 'eval_samples_per_second': 1165.605, 'eval_steps_per_second': 83.257, 'epoch': 20.0}


100%|██████████| 40/40 [01:09<00:00,  1.20s/it]

{'train_runtime': 69.3702, 'train_samples_per_second': 8.073, 'train_steps_per_second': 0.577, 'train_loss': 0.2898012806661427, 'epoch': 20.0}


2024/09/01 17:12:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run HooshvareLab/bert-base-parsbert-uncased at: http://127.0.0.1:5000/#/experiments/867072811883705549/runs/9f45d2840aeb4d228f1bfdd2039bd7ed.
2024/09/01 17:12:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/867072811883705549.
100%|██████████| 40/40 [01:10<00:00,  1.76s/it]


TrainOutput(global_step=40, training_loss=0.2898012806661427, metrics={'train_runtime': 69.3702, 'train_samples_per_second': 8.073, 'train_steps_per_second': 0.577, 'total_flos': 5630484866832.0, 'train_loss': 0.2898012806661427, 'epoch': 20.0})

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [46]:
text = "دلم خیلی درد می کنه"

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [47]:
from transformers import pipeline

classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-problem',
  'score': 0.9168041,
  'index': 1,
  'word': 'دلم',
  'start': 0,
  'end': 3},
 {'entity': 'I-problem',
  'score': 0.85156476,
  'index': 2,
  'word': 'خیلی',
  'start': 4,
  'end': 8},
 {'entity': 'I-problem',
  'score': 0.92179,
  'index': 3,
  'word': 'درد',
  'start': 9,
  'end': 12}]